In [1]:
import requests
import time
import tqdm
import pandas as pd
import os
import json
from bs4 import BeautifulSoup
from tqdm import tqdm

import re

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [2]:
dflinks = pd.read_csv('Alle_villa_kom.csv')
dflinks['URL']
links = dflinks['URL'].tolist()
print (links)

['https://www.boligsiden.dk/adresse/kongsager-32-2620-albertslund-01650275__32_______', 'https://www.boligsiden.dk/adresse/randager-73-2620-albertslund-01650370__73_______', 'https://www.boligsiden.dk/adresse/snebaerhaven-16-2620-albertslund-01650458__16_______', 'https://www.boligsiden.dk/adresse/kongsager-89-2620-albertslund-01650275__89_______', 'https://www.boligsiden.dk/adresse/flintager-9-2620-albertslund-01650107___9_______', 'https://www.boligsiden.dk/adresse/kastanie-alle-17a-2620-albertslund-01650260_17a_______', 'https://www.boligsiden.dk/adresse/toftestraede-4-2620-albertslund-01652665___4_______', 'https://www.boligsiden.dk/adresse/randager-111-2620-albertslund-01650370_111_______', 'https://www.boligsiden.dk/adresse/kongsager-4-2620-albertslund-01650275___4_______', 'https://www.boligsiden.dk/adresse/enebaerhaven-103-2620-albertslund-01650089_103_______', 'https://www.boligsiden.dk/adresse/porsager-58-2620-albertslund-01650365__58_______', 'https://www.boligsiden.dk/adres

In [3]:
nan_urls = dflinks[dflinks['URL'].isna()]
print(nan_urls)


Empty DataFrame
Columns: [URL, Adresse, Kort adresse, Kvdm, Latitude, Longitude, Kommune, Type, Nearest_Metro_Distance, Nearest_Tog_Distance]
Index: []


In [7]:
# import links and df in case of failure
running_links = pd.read_csv('running_links.csv')
links = running_links['links'].tolist()

running_df = pd.read_csv('running_df.csv')
df = running_df

In [7]:
df_without_duplicates = df.drop_duplicates(subset=['URL'], keep='first').reset_index(drop=True)
df_without_duplicates

,Date,Price,Status,Build Year,URL
0,,3600000,"Solgt, Fri handel",1967,https://www.boligsiden.dk/adresse/kongsager-32...
1,,2795000,Fjernet fra boligsiden,1965,https://www.boligsiden.dk/adresse/randager-73-...
2,,3695000,"Solgt, Fri handel",1969,https://www.boligsiden.dk/adresse/snebaerhaven...
3,,,Ombygning,1971,https://www.boligsiden.dk/adresse/kongsager-89...
4,,3450000,"Solgt, Fri handel",1965,https://www.boligsiden.dk/adresse/flintager-9-...
...,...,...,...,...,...
595,,2250000,"Solgt, Familiehandel",1981,https://www.boligsiden.dk/adresse/moellemosen-...
596,,3595000,Fjernet fra boligsiden,1997,https://www.boligsiden.dk/adresse/hoestbuen-83...
597,,3660000,Fjernet fra boligsiden,1972,https://www.boligsiden.dk/adresse/viggo-barfoe...
598,,3495000,Fjernet fra boligsiden,1969,https://www.boligsiden.dk/adresse/aage-berntse...


In [18]:
df = pd.read_csv('Alle_villa.csv')

In [6]:
len(links)

31169

In [4]:
df = pd.DataFrame ()

In [5]:
# Initialize index variable
i = 0
data_elements = []
max_links_per_iteration = 100

# Iterate through each URL and scrape the data
with tqdm(total=len(links)) as pbar:
    while i < len(links):
        # Get the URL
        url = links[i]

        try:
            # Send a GET request to the URL
            response = requests.get(url)

            # Check if the request was successful (status code 200)
            if response.status_code == 200:
                # Extract the page source HTML content
                html_content = response.text

                # Find the substring containing the "events" category within the HTML content
                start_index = html_content.find('"events":[')
                end_index = html_content.find(']}', start_index) + 2
                events_data_str = html_content[start_index:end_index]

                # Parse the substring to extract the necessary information
                events_data = []
                for event_str in events_data_str.split('},{'):
                    event = {}
                    for item in event_str.strip('{}').split(','):
                        parts = item.split(':', 1)
                        if len(parts) == 2:
                            key, value = parts
                            event[key.strip('"')] = value.strip('"')
                        elif len(parts) == 1:
                            key = parts[0]
                            event[key.strip('"')] = ""
                    events_data.append(event)

                # Extract the "Build Year" for the property
                build_year_str_match = re.search(r'"at":"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z","type":"built","title":"Bygget","label":"Byggeår"', html_content)
                if build_year_str_match:
                    build_year_str = build_year_str_match.group()
                    # Parse the JSON string to extract the "at" value
                    build_year_data = json.loads('{' + build_year_str + '}')
                    at_value = build_year_data.get("at", "")
                    # Extract the year part from the "at" value
                    build_year = at_value[:4] if at_value else ""
                else:
                    build_year = ""

                # Append the extracted information to the list of elements
                for event in events_data:
                    status = event.get("title", "")
                    if status == "Solgt":
                        status += ", " + event.get("label", "")
                    element = {
                        "Date": event.get("at", ""),
                        "Price": event.get("price", ""),
                        "Status": status,
                        "Build Year": build_year,
                        "URL": url
                    }
                    data_elements.append(element)

            else:
                print(f"Failed to retrieve data from {url}", flush=True)

        except requests.exceptions.MissingSchema:
            print(f"Invalid URL: {url}. Skipping...", flush=True)

        # Increment index
        i += 1
        pbar.update(1)  # Update tqdm progress bar

        # Check if it's time to merge and remove processed URLs
        if i % max_links_per_iteration == 0 or i == len(links):
            # Create a DataFrame from the list of elements
            df_temp = pd.DataFrame(data_elements)

            # Merge the DataFrame with the accumulated DataFrame
            df = pd.concat([df, df_temp], ignore_index=True)

            # Clear the data_elements list for the next iteration
            data_elements = []

            links = links[i:]

            # Reset the index for the next iteration
            i = 0
            # Convert the list of links to a DataFrame
            links_df = pd.DataFrame({'links': links})
            # Save the links DataFrame as a CSV file, overriding existing files
            links_df.to_csv('running_links.csv', index=False, mode='w')
            df.to_csv('running_df.csv', index=False, mode='w')

df.to_csv('Alle_villa.csv', index=False)


  0%|          | 0/31769 [00:00<?, ?it/s]

  2%|▏         | 602/31769 [05:02<4:21:08,  1.99it/s] 


KeyboardInterrupt: 

In [4]:
df

,Date,Price,Status,Build Year,URL
0,NaN,3600000.0,"Solgt, Fri handel",1967.0,https://www.boligsiden.dk/adresse/kongsager-32...
1,2020-12-29T07:07:16.203Z,3665000.0,Fjernet fra boligsiden,1967.0,https://www.boligsiden.dk/adresse/kongsager-32...
2,2020-12-28T13:54:53.393Z,3665000.0,Udbudt til salg på boligsiden,1967.0,https://www.boligsiden.dk/adresse/kongsager-32...
3,2020-12-22T16:44:51.543Z,3665000.0,Fjernet fra boligsiden,1967.0,https://www.boligsiden.dk/adresse/kongsager-32...
4,2020-11-25T23:00:00Z,3665000.0,Udbudt til salg på boligsiden,1967.0,https://www.boligsiden.dk/adresse/kongsager-32...
...,...,...,...,...,...
272395,2022-03-03T11:07:05.093Z,4495000,Fjernet fra boligsiden,1976,https://www.boligsiden.dk/adresse/vallensbaek-...
272396,2022-02-16T09:16:39Z,4495000,Udbudt til salg på boligsiden,1976,https://www.boligsiden.dk/adresse/vallensbaek-...
272397,2001-04-23T00:00:00Z,1919090,"Solgt, Fri handel",1976,https://www.boligsiden.dk/adresse/vallensbaek-...
272398,1993-04-21T00:00:00Z,954770,"Solgt, Fri handel",1976,https://www.boligsiden.dk/adresse/vallensbaek-...


In [20]:
# Fill missing and non-numeric values in 'Build Year' column with -1
df['Build Year'] = pd.to_numeric(df['Build Year'], errors='coerce').fillna(-1).astype(int)

# Reset the index
df.reset_index(drop=True, inplace=True)

# Print the cleaned DataFrame
print(df)


                       Date      Price             Status  Build Year  \
0                       NaN  3600000.0  Solgt, Fri handel        1967   
1      2018-08-22T00:00:00Z  1650000.0      Solgt, Øvrigt        1967   
2      2012-10-30T00:00:00Z  2495000.0  Solgt, Fri handel        1967   
3      2007-05-21T00:00:00Z  2625000.0  Solgt, Fri handel        1967   
4      2004-09-04T00:00:00Z  1600000.0  Solgt, Fri handel        1967   
...                     ...        ...                ...         ...   
75189                          1925000  Solgt, Fri handel        1970   
75190  2015-01-23T00:00:00Z    2700000  Solgt, Fri handel        1970   
75191  2022-05-04T00:00:00Z    4415000  Solgt, Fri handel        1976   
75192  2001-04-23T00:00:00Z    1919090  Solgt, Fri handel        1976   
75193  1993-04-21T00:00:00Z     954770  Solgt, Fri handel        1976   

                                                     URL  
0      https://www.boligsiden.dk/adresse/kongsager-32...  
1    

In [10]:
# Filter the DataFrame to show all rows where Build Year is -1
missing_build_years = df[df['Build Year'] == -1]
missing_build_years.to_csv('NoBuildYearAlle_villa.csv', index=False)
# Filter the DataFrame to remove all rows where Build Year is -1
df = df[df['Build Year'] != -1]
df


,Date,Price,Status,Build Year,URL
0,NaN,3600000.0,"Solgt, Fri handel",1967,https://www.boligsiden.dk/adresse/kongsager-32...
1,2018-08-22T00:00:00Z,1650000.0,"Solgt, Øvrigt",1967,https://www.boligsiden.dk/adresse/kongsager-32...
2,2012-10-30T00:00:00Z,2495000.0,"Solgt, Fri handel",1967,https://www.boligsiden.dk/adresse/kongsager-32...
3,2007-05-21T00:00:00Z,2625000.0,"Solgt, Fri handel",1967,https://www.boligsiden.dk/adresse/kongsager-32...
4,2004-09-04T00:00:00Z,1600000.0,"Solgt, Fri handel",1967,https://www.boligsiden.dk/adresse/kongsager-32...
...,...,...,...,...,...
75189,,1925000,"Solgt, Fri handel",1970,https://www.boligsiden.dk/adresse/syvhoejvaeng...
75190,2015-01-23T00:00:00Z,2700000,"Solgt, Fri handel",1970,https://www.boligsiden.dk/adresse/syvhoejvaeng...
75191,2022-05-04T00:00:00Z,4415000,"Solgt, Fri handel",1976,https://www.boligsiden.dk/adresse/vallensbaek-...
75192,2001-04-23T00:00:00Z,1919090,"Solgt, Fri handel",1976,https://www.boligsiden.dk/adresse/vallensbaek-...


In [11]:
# Filter rows with NaN values in the 'Status' column
rows_with_nan_status = df[df['Status'].isna()]

# Save rows with NaN status to a CSV file
rows_with_nan_status.to_csv('NANStatusAlle_villa.csv', index=False)

# Drop rows with missing values in the 'Status' column
df.dropna(subset=['Status'], inplace=True)

# Filter out rows where 'Status' does not include "Solgt"
df = df[df['Status'].str.contains('Solgt')]


In [12]:
# Create a copy of the DataFrame
df_copy = df.copy()

# Update Date column to remove time part and dates
df_copy.loc[:, 'Date'] = df_copy['Date'].str.split('T').str[0]  # Extract only the date part
df_copy.loc[:, 'Date'] = pd.to_datetime(df_copy['Date'])

# Filter the dataframe to keep only dates between 2015 and 2023
df_copy = df_copy[(df_copy['Date'].dt.year >= 2015) & (df_copy['Date'].dt.year <= 2023)]

# Reset the index of the DataFrame
df_copy.reset_index(drop=True, inplace=True)

# Print the cleaned DataFrame
print(df_copy)


            Date      Price                Status  Build Year  \
0     2018-08-22  1650000.0         Solgt, Øvrigt        1967   
1     2019-09-26  2700000.0     Solgt, Fri handel        1965   
2     2017-01-27  2650000.0     Solgt, Fri handel        1971   
3     2021-10-07  4495000.0     Solgt, Fri handel        1968   
4     2017-01-25  2500000.0     Solgt, Fri handel        1968   
...          ...        ...                   ...         ...   
29990 2018-01-16    4150000     Solgt, Fri handel        1964   
29991 2019-10-03    1750000  Solgt, Familiehandel        1956   
29992 2015-02-11    3460000     Solgt, Fri handel        2001   
29993 2015-01-23    2700000     Solgt, Fri handel        1970   
29994 2022-05-04    4415000     Solgt, Fri handel        1976   

                                                     URL  
0      https://www.boligsiden.dk/adresse/kongsager-32...  
1      https://www.boligsiden.dk/adresse/randager-73-...  
2      https://www.boligsiden.dk/adresse/k

C:\Users\jge19\AppData\Local\Temp\ipykernel_10712\1137713455.py:6: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_copy.loc[:, 'Date'] = pd.to_datetime(df_copy['Date'])


In [13]:
nan_prices = df_copy[df_copy['Price'].isna()]
print(nan_prices)


Empty DataFrame
Columns: [Date, Price, Status, Build Year, URL]
Index: []


,Date,Price,Status,Build Year,URL
0,2019-02-24,2375000,"Solgt, Fri handel",1969,https://www.boligsiden.dk/adresse/elmehusene-1...
1,2023-12-08,3550000,"Solgt, Fri handel",2014,https://www.boligsiden.dk/adresse/tornegaardsv...
2,2017-11-06,2850000,"Solgt, Fri handel",2014,https://www.boligsiden.dk/adresse/tornegaardsv...
3,2022-08-22,3495000,"Solgt, Fri handel",1971,https://www.boligsiden.dk/adresse/humlehusene-...
4,2016-10-17,2795000,"Solgt, Fri handel",1971,https://www.boligsiden.dk/adresse/humlehusene-...
...,...,...,...,...,...
42816,2018-01-16,4150000,"Solgt, Fri handel",1964,https://www.boligsiden.dk/adresse/hyttebovej-3...
42817,2019-10-03,1750000,"Solgt, Familiehandel",1956,https://www.boligsiden.dk/adresse/bygaden-34-2...
42818,2015-02-11,3460000,"Solgt, Fri handel",2001,https://www.boligsiden.dk/adresse/nordmarksvae...
42819,2015-01-23,2700000,"Solgt, Fri handel",1970,https://www.boligsiden.dk/adresse/syvhoejvaeng...


In [14]:
df_copy.loc[:, 'Price'] = df_copy['Price'].astype(int)
df_copy


C:\Users\jge19\AppData\Local\Temp\ipykernel_10712\2437176605.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_copy.loc[:, 'Price'] = df_copy['Price'].astype(int)


,Date,Price,Status,Build Year,URL
0,2018-08-22,1650000,"Solgt, Øvrigt",1967,https://www.boligsiden.dk/adresse/kongsager-32...
1,2019-09-26,2700000,"Solgt, Fri handel",1965,https://www.boligsiden.dk/adresse/randager-73-...
2,2017-01-27,2650000,"Solgt, Fri handel",1971,https://www.boligsiden.dk/adresse/kongsager-89...
3,2021-10-07,4495000,"Solgt, Fri handel",1968,https://www.boligsiden.dk/adresse/randager-111...
4,2017-01-25,2500000,"Solgt, Fri handel",1968,https://www.boligsiden.dk/adresse/randager-111...
...,...,...,...,...,...
29990,2018-01-16,4150000,"Solgt, Fri handel",1964,https://www.boligsiden.dk/adresse/hyttebovej-3...
29991,2019-10-03,1750000,"Solgt, Familiehandel",1956,https://www.boligsiden.dk/adresse/bygaden-34-2...
29992,2015-02-11,3460000,"Solgt, Fri handel",2001,https://www.boligsiden.dk/adresse/nordmarksvae...
29993,2015-01-23,2700000,"Solgt, Fri handel",1970,https://www.boligsiden.dk/adresse/syvhoejvaeng...


In [17]:
# Find URLs present only in df_copy
urls_in_df_copy = set(df_copy['URL'])
urls_only_in_df_copy = urls_in_df_copy - set(dflinks['URL'])

# Find URLs present only in dflinks
urls_in_dflinks = set(dflinks['URL'])
urls_only_in_dflinks = urls_in_dflinks - urls_in_df_copy

print(len(urls_only_in_df_copy))
len(urls_only_in_dflinks)


10106


6853

In [15]:
dflinks = pd.read_csv('Alle_villa_kom.csv')

combinedf = pd.merge(df_copy, dflinks, on='URL', how='left')
combinedf = combinedf.dropna(subset=['Date'])
combinedf

,Date,Price,Status,Build Year,URL,Adresse,Kort adresse,Kvdm,Latitude,Longitude,Kommune,Type,Nearest_Metro_Distance,Nearest_Tog_Distance
0,2018-08-22,1650000,"Solgt, Øvrigt",1967,https://www.boligsiden.dk/adresse/kongsager-32...,Kongsager 32 2620 Albertslund,Kongsager 32 2620,131.0,55.665544,12.335459,Albertslund,Villa,9.569062,1.411333
1,2019-09-26,2700000,"Solgt, Fri handel",1965,https://www.boligsiden.dk/adresse/randager-73-...,Randager 73 2620 Albertslund,Randager 73 2620,104.0,55.667871,12.333803,Albertslund,Villa,9.594694,1.653715
2,2017-01-27,2650000,"Solgt, Fri handel",1971,https://www.boligsiden.dk/adresse/kongsager-89...,Kongsager 89 2620 Albertslund,Kongsager 89 2620,124.0,55.664092,12.333272,Albertslund,Villa,9.748918,1.443700
3,2021-10-07,4495000,"Solgt, Fri handel",1968,https://www.boligsiden.dk/adresse/randager-111...,Randager 111 2620 Albertslund,Randager 111 2620,140.0,55.668317,12.331590,Albertslund,Villa,9.715151,1.791785
4,2017-01-25,2500000,"Solgt, Fri handel",1968,https://www.boligsiden.dk/adresse/randager-111...,Randager 111 2620 Albertslund,Randager 111 2620,140.0,55.668317,12.331590,Albertslund,Villa,9.715151,1.791785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29990,2018-01-16,4150000,"Solgt, Fri handel",1964,https://www.boligsiden.dk/adresse/hyttebovej-3...,Hyttebovej 33 2665 Vallensbæk Strand,Hyttebovej 33 2665,138.0,55.621099,12.398445,Vallensbæk,Villa,9.366192,0.672066
29991,2019-10-03,1750000,"Solgt, Familiehandel",1956,https://www.boligsiden.dk/adresse/bygaden-34-2...,Bygaden 34 2625 Vallensbæk,Bygaden 34 2625,101.0,55.636328,12.363760,Vallensbæk,Villa,9.563621,2.124783
29992,2015-02-11,3460000,"Solgt, Fri handel",2001,https://www.boligsiden.dk/adresse/nordmarksvae...,Nordmarksvænge 77 2625 Vallensbæk,Nordmarksvænge 77 2625,130.0,55.646676,12.370577,Vallensbæk,Villa,8.521281,1.654918
29993,2015-01-23,2700000,"Solgt, Fri handel",1970,https://www.boligsiden.dk/adresse/syvhoejvaeng...,Syvhøjvænge 20 2625 Vallensbæk,Syvhøjvænge 20 2625,227.0,55.644262,12.370488,Vallensbæk,Villa,8.684703,1.865999


In [16]:
combinedf.dropna(subset=['Type'], inplace=True)
combinedf

,Date,Price,Status,Build Year,URL,Adresse,Kort adresse,Kvdm,Latitude,Longitude,Kommune,Type,Nearest_Metro_Distance,Nearest_Tog_Distance
0,2018-08-22,1650000,"Solgt, Øvrigt",1967,https://www.boligsiden.dk/adresse/kongsager-32...,Kongsager 32 2620 Albertslund,Kongsager 32 2620,131.0,55.665544,12.335459,Albertslund,Villa,9.569062,1.411333
1,2019-09-26,2700000,"Solgt, Fri handel",1965,https://www.boligsiden.dk/adresse/randager-73-...,Randager 73 2620 Albertslund,Randager 73 2620,104.0,55.667871,12.333803,Albertslund,Villa,9.594694,1.653715
2,2017-01-27,2650000,"Solgt, Fri handel",1971,https://www.boligsiden.dk/adresse/kongsager-89...,Kongsager 89 2620 Albertslund,Kongsager 89 2620,124.0,55.664092,12.333272,Albertslund,Villa,9.748918,1.443700
3,2021-10-07,4495000,"Solgt, Fri handel",1968,https://www.boligsiden.dk/adresse/randager-111...,Randager 111 2620 Albertslund,Randager 111 2620,140.0,55.668317,12.331590,Albertslund,Villa,9.715151,1.791785
4,2017-01-25,2500000,"Solgt, Fri handel",1968,https://www.boligsiden.dk/adresse/randager-111...,Randager 111 2620 Albertslund,Randager 111 2620,140.0,55.668317,12.331590,Albertslund,Villa,9.715151,1.791785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29990,2018-01-16,4150000,"Solgt, Fri handel",1964,https://www.boligsiden.dk/adresse/hyttebovej-3...,Hyttebovej 33 2665 Vallensbæk Strand,Hyttebovej 33 2665,138.0,55.621099,12.398445,Vallensbæk,Villa,9.366192,0.672066
29991,2019-10-03,1750000,"Solgt, Familiehandel",1956,https://www.boligsiden.dk/adresse/bygaden-34-2...,Bygaden 34 2625 Vallensbæk,Bygaden 34 2625,101.0,55.636328,12.363760,Vallensbæk,Villa,9.563621,2.124783
29992,2015-02-11,3460000,"Solgt, Fri handel",2001,https://www.boligsiden.dk/adresse/nordmarksvae...,Nordmarksvænge 77 2625 Vallensbæk,Nordmarksvænge 77 2625,130.0,55.646676,12.370577,Vallensbæk,Villa,8.521281,1.654918
29993,2015-01-23,2700000,"Solgt, Fri handel",1970,https://www.boligsiden.dk/adresse/syvhoejvaeng...,Syvhøjvænge 20 2625 Vallensbæk,Syvhøjvænge 20 2625,227.0,55.644262,12.370488,Vallensbæk,Villa,8.684703,1.865999


In [21]:
random_rows = df.sample(n=10, random_state=42)  # Setting random_state to 42
random_rows

,Date,Price,Status,Build Year,URL
9489,2007-08-02T00:00:00Z,2625000.0,"Solgt, Fri handel",1960,https://www.boligsiden.dk/adresse/ganloesevej-...
17657,2017-07-13T00:00:00Z,9100000.0,"Solgt, Fri handel",1921,https://www.boligsiden.dk/adresse/christiansve...
23757,2021-04-07T00:00:00Z,11000000.0,"Solgt, Fri handel",1889,https://www.boligsiden.dk/adresse/trunnevangen...
56995,2007-10-31T00:00:00Z,2900000.0,"Solgt, Fri handel",1935,https://www.boligsiden.dk/adresse/pildamsvej-2...
64474,2020-02-04T00:00:00Z,4400000,"Solgt, Fri handel",2021,https://www.boligsiden.dk/adresse/skovlybakken...
16528,1997-09-09T00:00:00Z,3858691.0,"Solgt, Fri handel",1930,https://www.boligsiden.dk/adresse/ole-olsens-a...
37108,2000-09-29T00:00:00Z,1925000.0,"Solgt, Fri handel",1988,https://www.boligsiden.dk/adresse/olsbaekeng-2...
57192,1994-04-08T00:00:00Z,726440.0,"Solgt, Fri handel",1939,https://www.boligsiden.dk/adresse/roedager-all...
26689,2021-04-28T00:00:00Z,8695000.0,"Solgt, Fri handel",2014,https://www.boligsiden.dk/adresse/hoejmarksvej...
13447,1992-01-27T00:00:00Z,355000.0,"Solgt, Familiehandel",1952,https://www.boligsiden.dk/adresse/farum-gydeve...


In [17]:
# Remove duplicate URLs from dflinks dataframe
dflinks_unique = dflinks.drop_duplicates(subset=['URL'])

# Perform the merge after removing duplicates
combined_df = pd.merge(df_copy, dflinks_unique, on='URL', how='left')

# Reset the index of the DataFrame
combined_df.reset_index(drop=True, inplace=True)

# Print the combined DataFrame
combined_df


,Date,Price,Status,Build Year,URL,Adresse,Kort adresse,Kvdm,Latitude,Longitude,Kommune,Type,Nearest_Metro_Distance,Nearest_Tog_Distance
0,2018-08-22,1650000,"Solgt, Øvrigt",1967,https://www.boligsiden.dk/adresse/kongsager-32...,Kongsager 32 2620 Albertslund,Kongsager 32 2620,131.0,55.665544,12.335459,Albertslund,Villa,9.569062,1.411333
1,2019-09-26,2700000,"Solgt, Fri handel",1965,https://www.boligsiden.dk/adresse/randager-73-...,Randager 73 2620 Albertslund,Randager 73 2620,104.0,55.667871,12.333803,Albertslund,Villa,9.594694,1.653715
2,2017-01-27,2650000,"Solgt, Fri handel",1971,https://www.boligsiden.dk/adresse/kongsager-89...,Kongsager 89 2620 Albertslund,Kongsager 89 2620,124.0,55.664092,12.333272,Albertslund,Villa,9.748918,1.443700
3,2021-10-07,4495000,"Solgt, Fri handel",1968,https://www.boligsiden.dk/adresse/randager-111...,Randager 111 2620 Albertslund,Randager 111 2620,140.0,55.668317,12.331590,Albertslund,Villa,9.715151,1.791785
4,2017-01-25,2500000,"Solgt, Fri handel",1968,https://www.boligsiden.dk/adresse/randager-111...,Randager 111 2620 Albertslund,Randager 111 2620,140.0,55.668317,12.331590,Albertslund,Villa,9.715151,1.791785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29990,2018-01-16,4150000,"Solgt, Fri handel",1964,https://www.boligsiden.dk/adresse/hyttebovej-3...,Hyttebovej 33 2665 Vallensbæk Strand,Hyttebovej 33 2665,138.0,55.621099,12.398445,Vallensbæk,Villa,9.366192,0.672066
29991,2019-10-03,1750000,"Solgt, Familiehandel",1956,https://www.boligsiden.dk/adresse/bygaden-34-2...,Bygaden 34 2625 Vallensbæk,Bygaden 34 2625,101.0,55.636328,12.363760,Vallensbæk,Villa,9.563621,2.124783
29992,2015-02-11,3460000,"Solgt, Fri handel",2001,https://www.boligsiden.dk/adresse/nordmarksvae...,Nordmarksvænge 77 2625 Vallensbæk,Nordmarksvænge 77 2625,130.0,55.646676,12.370577,Vallensbæk,Villa,8.521281,1.654918
29993,2015-01-23,2700000,"Solgt, Fri handel",1970,https://www.boligsiden.dk/adresse/syvhoejvaeng...,Syvhøjvænge 20 2625 Vallensbæk,Syvhøjvænge 20 2625,227.0,55.644262,12.370488,Vallensbæk,Villa,8.684703,1.865999


In [18]:
# Filter out rows where 'Status' does not include "Fri handel"
filtered_rows = combined_df[~combined_df['Status'].str.contains('Fri handel')]

# Save the filtered rows to a CSV file
filtered_rows.to_csv('IkkeFriHandelAlle_villa.csv', index=False)

# Keep only rows where 'Status' includes "Fri handel"
combined_df = combined_df[combined_df['Status'].str.contains('Fri handel')]

# Reset the index of the DataFrame
combined_df.reset_index(drop=True, inplace=True)

# Print the updated DataFrame
combined_df


,Date,Price,Status,Build Year,URL,Adresse,Kort adresse,Kvdm,Latitude,Longitude,Kommune,Type,Nearest_Metro_Distance,Nearest_Tog_Distance
0,2019-09-26,2700000,"Solgt, Fri handel",1965,https://www.boligsiden.dk/adresse/randager-73-...,Randager 73 2620 Albertslund,Randager 73 2620,104.0,55.667871,12.333803,Albertslund,Villa,9.594694,1.653715
1,2017-01-27,2650000,"Solgt, Fri handel",1971,https://www.boligsiden.dk/adresse/kongsager-89...,Kongsager 89 2620 Albertslund,Kongsager 89 2620,124.0,55.664092,12.333272,Albertslund,Villa,9.748918,1.443700
2,2021-10-07,4495000,"Solgt, Fri handel",1968,https://www.boligsiden.dk/adresse/randager-111...,Randager 111 2620 Albertslund,Randager 111 2620,140.0,55.668317,12.331590,Albertslund,Villa,9.715151,1.791785
3,2017-01-25,2500000,"Solgt, Fri handel",1968,https://www.boligsiden.dk/adresse/randager-111...,Randager 111 2620 Albertslund,Randager 111 2620,140.0,55.668317,12.331590,Albertslund,Villa,9.715151,1.791785
4,2017-01-16,2125000,"Solgt, Fri handel",1968,https://www.boligsiden.dk/adresse/randager-111...,Randager 111 2620 Albertslund,Randager 111 2620,140.0,55.668317,12.331590,Albertslund,Villa,9.715151,1.791785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28882,2019-01-29,3675000,"Solgt, Fri handel",1969,https://www.boligsiden.dk/adresse/syvhoejvaeng...,Syvhøjvænge 34 2625 Vallensbæk,Syvhøjvænge 34 2625,186.0,55.645060,12.370012,Vallensbæk,Villa,8.655602,1.776132
28883,2018-01-16,4150000,"Solgt, Fri handel",1964,https://www.boligsiden.dk/adresse/hyttebovej-3...,Hyttebovej 33 2665 Vallensbæk Strand,Hyttebovej 33 2665,138.0,55.621099,12.398445,Vallensbæk,Villa,9.366192,0.672066
28884,2015-02-11,3460000,"Solgt, Fri handel",2001,https://www.boligsiden.dk/adresse/nordmarksvae...,Nordmarksvænge 77 2625 Vallensbæk,Nordmarksvænge 77 2625,130.0,55.646676,12.370577,Vallensbæk,Villa,8.521281,1.654918
28885,2015-01-23,2700000,"Solgt, Fri handel",1970,https://www.boligsiden.dk/adresse/syvhoejvaeng...,Syvhøjvænge 20 2625 Vallensbæk,Syvhøjvænge 20 2625,227.0,55.644262,12.370488,Vallensbæk,Villa,8.684703,1.865999


In [19]:
combined_df.to_csv('Alle_villa_færdig.csv', index=False)

In [38]:
#*******************************************************************************************
EjerLej = pd.read_csv('Alle_ejerlejlighed_færdig.csv')
VillaRækkeCPH = pd.read_csv('CPHVilla_Række_færdig.csv')
UdenCPHRække = pd.read_csv('Alle-rækkehus_færdig.csv')
UdenCPHVilla = pd.read_csv('Alle_villa_færdig.csv')

In [39]:
EjerLej['Type'] = 'Ejerlejlighed'


In [40]:
# Concatenate the two DataFrames along the rows axis
combined_Done_df = pd.concat([EjerLej, VillaRækkeCPH,UdenCPHRække,UdenCPHVilla], ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_Done_df.to_csv('DATA.csv', index=False)

In [41]:
combined_Done_df

,Date,Price,Status,Build Year,URL,Adresse,Kort adresse,Kvdm,Latitude,Longitude,Kommune,Type,Nearest_Metro_Distance,Nearest_Tog_Distance
0,2018-10-29,1845000,"Solgt, Fri handel",1933,https://www.boligsiden.dk/adresse/elmelundevej-22-1-th-2700-broenshoej-01011508__22__1__th,"Elmelundevej 22, 1. th. 2700 Brønshøj",Elmelundevej 22,60.0,55.702660,12.496368,København,Ejerlejlighed,1.589281,1.740477
1,2017-07-06,1400000,"Solgt, Fri handel",1933,https://www.boligsiden.dk/adresse/elmelundevej-22-1-th-2700-broenshoej-01011508__22__1__th,"Elmelundevej 22, 1. th. 2700 Brønshøj",Elmelundevej 22,60.0,55.702660,12.496368,København,Ejerlejlighed,1.589281,1.740477
2,2015-06-26,101633750,"Solgt, Fri handel""}]",2017,https://www.boligsiden.dk/adresse/alliancevej-23-0-th-2450-koebenhavn-sv-01010090__23_st__th,"Alliancevej 23, st. th. 2450 København SV",Alliancevej 23,93.0,55.648767,12.553162,København,Ejerlejlighed,2.006673,1.189159
3,2019-06-25,3225000,"Solgt, Fri handel",2006,https://www.boligsiden.dk/adresse/weidekampsgade-57-5-3-2300-koebenhavn-s-01018122__57__5___3,"Weidekampsgade 57, 5. 3. 2300 København S",Weidekampsgade 57,67.0,55.667245,12.581967,København,Ejerlejlighed,0.459682,1.303566
4,2023-05-02,2600000,"Solgt, Fri handel",1935,https://www.boligsiden.dk/adresse/vestbanevej-15-4-th-2500-valby-01018196__15__4__th,"Vestbanevej 15, 4. th. 2500 Valby",Vestbanevej 15,65.0,55.664748,12.521148,København,Ejerlejlighed,1.329182,0.429430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125380,2019-01-29,3675000,"Solgt, Fri handel",1969,https://www.boligsiden.dk/adresse/syvhoejvaenge-34-2625-vallensbaek-01877882__34_______,Syvhøjvænge 34 2625 Vallensbæk,Syvhøjvænge 34 2625,186.0,55.645060,12.370012,Vallensbæk,Villa,8.655602,1.776132
125381,2018-01-16,4150000,"Solgt, Fri handel",1964,https://www.boligsiden.dk/adresse/hyttebovej-33-2665-vallensbaek-strand-01872769__33_______,Hyttebovej 33 2665 Vallensbæk Strand,Hyttebovej 33 2665,138.0,55.621099,12.398445,Vallensbæk,Villa,9.366192,0.672066
125382,2015-02-11,3460000,"Solgt, Fri handel",2001,https://www.boligsiden.dk/adresse/nordmarksvaenge-77-2625-vallensbaek-01875600__77_______,Nordmarksvænge 77 2625 Vallensbæk,Nordmarksvænge 77 2625,130.0,55.646676,12.370577,Vallensbæk,Villa,8.521281,1.654918
125383,2015-01-23,2700000,"Solgt, Fri handel",1970,https://www.boligsiden.dk/adresse/syvhoejvaenge-20-2625-vallensbaek-01877882__20_______,Syvhøjvænge 20 2625 Vallensbæk,Syvhøjvænge 20 2625,227.0,55.644262,12.370488,Vallensbæk,Villa,8.684703,1.865999


In [42]:
pd.set_option('display.max_colwidth', None)  # Set display option to show all column contents
combined_Done_df.sample(n=10)

,Date,Price,Status,Build Year,URL,Adresse,Kort adresse,Kvdm,Latitude,Longitude,Kommune,Type,Nearest_Metro_Distance,Nearest_Tog_Distance
49016,2021-08-15,5150000,"Solgt, Fri handel",1903,https://www.boligsiden.dk/adresse/rosenvaengets-alle-17b-2-th-2100-koebenhavn-oe-01015948_17b__2__th,"Rosenvængets Allé 17B, 2. th. 2100 København Ø",Rosenvængets Allé 17,68.0,55.699306,12.580845,København,Ejerlejlighed,0.248445,0.852002
79599,2016-04-26,6798000,"Solgt, Fri handel",1917,https://www.boligsiden.dk/adresse/strindbergsvej-46-2500-valby-01017028__46_______,Strindbergsvej 46 2500 Valby,Strindbergsvej 46 2500,176.0,55.656112,12.521033,København,Villa,NaN,NaN
35185,2021-08-31,4600000,"Solgt, Fri handel",2006,https://www.boligsiden.dk/adresse/amerika-plads-26b-5-4-2100-koebenhavn-oe-01010139_26b__5___4,"Amerika Plads 26B, 5. 4. 2100 København Ø",Amerika Plads 26,72.0,55.699649,12.594036,København,Ejerlejlighed,0.893516,0.661940
72828,2020-12-01,1930000,"Solgt, Fri handel",1959,https://www.boligsiden.dk/adresse/brandholms-alle-32b-2-2610-roedovre-01750085_32b__2____,"Brandholms Alle 32B, 2. 2610 Rødovre",Brandholms Alle 32,58.0,55.675416,12.449717,roedovre,Ejerlejlighed,2.640315,1.295854
117398,2018-03-08,4500000,"Solgt, Fri handel",1988,https://www.boligsiden.dk/adresse/skodsborgvej-21a-2830-virum-01730715_21a_______,Skodsborgvej 21A 2830 Virum,Skodsborgvej 21A 2830,193.0,55.797855,12.484591,Lyngby-Taarbæk,Villa,11.245166,0.742386
36304,2020-08-07,2100000,"Solgt, Fri handel",1929,https://www.boligsiden.dk/adresse/ungarnsgade-66-0-th-2300-koebenhavn-s-01017848__66_st__th,"Ungarnsgade 66, st. th. 2300 København S",Ungarnsgade 66,56.0,55.662759,12.614171,København,Ejerlejlighed,0.145324,3.234312
74494,2017-08-18,3025000,"Solgt, Fri handel",1956,https://www.boligsiden.dk/adresse/soelleroed-park-19-1-9-2840-holte-02300623__19__1___9,"Søllerød Park 19, 1. 9. Søllerød, 2840 Holte",Søllerød Park 19,105.0,55.811328,12.506393,Rudersdal,Ejerlejlighed,12.173243,2.696218
52093,2023-03-09,3350000,"Solgt, Fri handel",1973,https://www.boligsiden.dk/adresse/grundtvigsvej-4-0-tv-1864-frederiksberg-c-01470328___4_st__tv,"Grundtvigsvej 4, st. tv. 1864 Frederiksberg C",Grundtvigsvej 4,66.0,55.678053,12.541416,frederiksberg,Ejerlejlighed,0.651570,1.315757
105919,2015-09-08,3200000,"Solgt, Fri handel",1954,https://www.boligsiden.dk/adresse/langemosevej-29-2880-bagsvaerd-01590495__29_______,Langemosevej 29 2880 Bagsværd,Langemosevej 29 2880,157.0,55.758555,12.422616,Gladsaxe,Villa,8.334034,1.026780
86411,2020-10-24,5795000,"Solgt, Fri handel""}]",2021,https://www.boligsiden.dk/adresse/sletten-21-3500-vaerloese-01900786__21_______,"Sletten 21 Kirke Værløse, 3500 Værløse",Sletten 21,167.0,NaN,NaN,Furesoe,Rækkehus,inf,inf


In [44]:
import matplotlib.pyplot as plt
import seaborn as sns

In [54]:
df_DONE = pd.read_csv('mangler_kun_nik.csv')

In [59]:
# Define colors for each type
type_colors = {'Ejerlejlighed': 'blue', 'Villa': 'green', 'Rækkehus': 'red'}

# Create a FacetGrid with a plot for each type
g = sns.FacetGrid(df_DONE, col="Type", hue="Type", palette=type_colors.values(), col_wrap=3)
g.map(sns.scatterplot, "Price", "Kvdm", alpha=0.7)
g.set_titles("{col_name}")
g.set_xlabels("Price")
g.set_ylabels("Kvdm")
g.fig.subplots_adjust(top=0.9)  # Adjust the subplot layout
g.fig.suptitle('Scatter Plots of Prices with Different Types')

# Get the current Axes object from the FacetGrid
axes = g.axes.flatten()

# Set y-axis ticks and labels to display Kvdm values
for ax in axes:
    ax.set_yticks(df_DONE['Kvdm'])  # Set y-ticks to Kvdm values
    ax.set_yticklabels(df_DONE['Price'], rotation=45)  # Set y-tick labels to Price values
    ax.tick_params(axis='y', labelsize=8)  # Adjust y-axis tick label size

plt.show()


KeyboardInterrupt: 

In [48]:
# Check the data type of the 'Price' column
print(combined_Done_df['Price'].dtype)

# Convert 'Price' column to numeric if it's not already
combined_Done_df['Price'] = pd.to_numeric(combined_Done_df['Price'], errors='coerce')

# Check the data type again
print(combined_Done_df['Price'].dtype)


int64
int64
